# Milestone 2

In [30]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
import urllib.parse

## Step 1 - Setup EC2 Instance

https://github.com/UBC-MDS/DSCI525_group16/blob/main/img/step1.png

## Step 2 - Setup Server

https://github.com/UBC-MDS/DSCI525_group16/blob/main/img/step2.png

## Step 3 - Setup JupyterHub

https://github.com/UBC-MDS/DSCI525_group16/blob/main/img/step3.png

## Step 4 - Get Data

In [26]:
# Use API to download data
article_id = 14226968 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/srv/data/shared_data/"

In [27]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

In [28]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]   
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [29]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

## Step 5 - Setup S3 Bucket

https://github.com/UBC-MDS/DSCI525_group16/blob/main/img/step5.png

## Step 6 - Wrangle Data

In [5]:
import os
import pandas as pd
os.environ["AWS_SHARED_CREDENTIALS_FILE"] = "/srv/keys/credentials"

In [6]:
%%time
df = pd.read_parquet("s3://mds-s3-16-austin/combined_model_data_parti.parquet")
df

NoCredentialsError: Unable to locate credentials